# OmniMarket System with Ticker and Expiration Picker
Now supports interactive selection of stock ticker and available option expiry dates.

In [ ]:

# === Ticker and Expiry Selection ===
import yfinance as yf

# Ask user to enter a stock ticker
ticker = input("Enter stock ticker (e.g., AAPL, TSLA, MSFT): ").upper()
yf_ticker = yf.Ticker(ticker)

# Show available expiration dates
expirations = yf_ticker.options
if not expirations:
    raise ValueError(f"No options available for {ticker}. Please try another ticker.")

print(f"Available Expiration Dates for {ticker}:")
for i, date in enumerate(expirations):
    print(f"{i}: {date}")

# Ask user to select expiration date
expiry = input("Select expiration date (e.g., 2024-06-21): ")
if expiry not in expirations:
    raise ValueError("Invalid expiration date. Please copy exactly from the list above.")


In [ ]:

# === Stock Search Input ===
ticker = input("Enter stock ticker (e.g., AAPL, TSLA, MSFT): ").upper()


In [ ]:
# ===# OmniMarket Full System - Prediction + ML Preparation

In [ ]:
# ===Imports ===
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.stats import norm

In [ ]:
# ===Black-Scholes Pricing ===
def black_scholes_price(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [ ]:
# ===Greeks Calculation ===
def black_scholes_greeks(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    delta = norm.cdf(d1) if option_type == 'call' else -norm.cdf(-d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100
    theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) -
             r * K * np.exp(-r * T) * (norm.cdf(d2) if option_type == 'call' else norm.cdf(-d2))) / 365
    rho = (K * T * np.exp(-r * T) * (norm.cdf(d2) if option_type == 'call' else -norm.cdf(-d2))) / 100
    return {'Delta': delta, 'Gamma': gamma, 'Vega': vega, 'Theta': theta, 'Rho': rho}

In [ ]:
# ===Option Chain Loader ===
def get_option_chain(ticker, expiry):
    return yf.Ticker(ticker).option_chain(expiry)

In [ ]:
# ===High Accuracy Simulation ===
def simulate_option_accurate(ticker, expiry, call_index=10, put_index=10, r=0.05):
    chain = get_option_chain(ticker, expiry)
    calls, puts = chain.calls, chain.puts
    call = calls.iloc[call_index]
    put = puts.iloc[put_index]

    S0 = yf.Ticker(ticker).history(period='1d')['Close'].iloc[-1]
    expiry_date = pd.to_datetime(expiry)
    T = max((expiry_date - datetime.utcnow()).days / 365, 1/365)

    price_range = np.linspace(S0 * 0.7, S0 * 1.3, 150)
    call_premium = (call['bid'] + call['ask']) / 2
    put_premium = (put['bid'] + put['ask']) / 2
    call_iv = call.get('impliedVolatility', 0.3)
    put_iv = put.get('impliedVolatility', 0.3)

    call_bs_prices = [black_scholes_price(S, call.strike, T, r, call_iv, 'call') for S in price_range]
    put_bs_prices = [black_scholes_price(S, put.strike, T, r, put_iv, 'put') for S in price_range]
    call_pnl = np.array(call_bs_prices) - call_premium
    put_pnl = np.array(put_bs_prices) - put_premium

    plt.figure(figsize=(12, 6))
    plt.plot(price_range, call_pnl, label=f'Call @ {call.strike}')
    plt.plot(price_range, put_pnl, label=f'Put @ {put.strike}')
    plt.axhline(0, color='black', linestyle='--')
    plt.title(f"{ticker} Option P&L using Black-Scholes ({expiry})")
    plt.xlabel("Stock Price at Expiry")
    plt.ylabel("Profit / Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return {
        "Call Greeks": black_scholes_greeks(S0, call.strike, T, r, call_iv, 'call'),
        "Put Greeks": black_scholes_greeks(S0, put.strike, T, r, put_iv, 'put'),
        "Days to Expiry": T * 365,
        "Underlying Price": S0,
        "Call Premium": call_premium,
        "Put Premium": put_premium
    }

In [ ]:
# ===ML Training Base: Fetch Stock Data ===
def fetch_stock_data(ticker, months=6):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=30 * months)
    data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    return data[['Open', 'High', 'Low', 'Close', 'Volume']]

In [ ]:
# ===Example Runs ===
if __name__ == "__main__":
    # Run simulation
    print("Running prediction model...")
    sim_result = simulate_option_accurate(ticker, expiry)
    print(sim_result)

    # Fetch stock data for ML
    print("\nFetching stock data for ML training...")
    stock_df = fetch_stock_data(ticker)
    print(stock_df.head())